In [4]:
from bs4 import BeautifulSoup
import six.moves.urllib as urllib2
import re
import requests
import csv
import time

# Get all of the Urls

In [6]:
import pandas as pd;
import numpy as np;
import matplotlib.pyplot as plt;
import grequests;

/Users/bcipollini/code/venv3/lib/python3.7/site-packages/grequests.py:21: MonkeyPatchWarning: Monkey-patching ssl after ssl has already been imported may lead to errors, including RecursionError on Python 3.6. It may also silently lead to incorrect behaviour on Python 3.7. Please monkey-patch earlier. See https://github.com/gevent/gevent/issues/1016. Modules that had direct imports (NOT patched): ['urllib3.contrib.pyopenssl (/Users/bcipollini/code/venv3/lib/python3.7/site-packages/urllib3/contrib/pyopenssl.py)']. 
  curious_george.patch_all(thread=False, select=False)


In [10]:
data = pd.read_csv("Sample_Program_Service_Data.csv", encoding='latin')

/Users/bcipollini/code/venv3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (52,53,55,56,58,59) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
data.columns

Index(['Unnamed: 0', 'EIN', 'GROSSRECEIPTS', 'TOTEMPLOYEE', 'TOTVOLUNTEERS',
       'CITY', 'STATE', 'FISYR', 'TOTALPROGSERVEXP', 'FORMTYPE', 'FORMYEAR',
       'NAME', 'DBA', 'ADDRESS', 'ZIP', 'WEBSITE', 'URL', 'NteeFinal',
       'MISSION', 'DISCOPS', 'CONTRIBCURRENT', 'PSRCURRENT', 'INVINCCURRENT',
       'OTHERREVCURRENT', 'TOTALREVCURRENT', 'MEMBERDUES', 'GROSSSALESOTHER',
       'SALESCOSTOTHER', 'NETSALESOTHER', 'GROSSINCGAMING',
       'GROSSINCFNDEVENTS', 'NETSALESINV', 'GRANTSPAIDCURRENT',
       'MEMBERBENCURRENT', 'SALARIESCURRENT', 'TOTALEXPCURRENT',
       'REVLESSEXPCURRENT', 'TOTALASSETSBEGYEAR', 'TOTALASSETSENDYEAR',
       'TOTALLIABBEGYEAR', 'TOTALLIABENDYEAR', 'NETASSETSBEGYEAR',
       'OTHERASSETSCHANGES', 'NETASSETSENDYEAR', 'CASHINVBEGYEAR',
       'CASHINVENDYEAR', 'LANDBEGYEAR', 'LANDENDYEAR', 'OTHERASSETSBEGYEAR',
       'OTHERASSETSENDYEAR', 'LOBBYING', 'proj1Desc', 'proj1ExpAmt',
       'proj1GrantAmt', 'proj2Desc', 'proj2ExpAmt', 'proj2GrantAmt',
       'p

In [12]:
data.head(5)[["WEBSITE", "NAME", "ADDRESS", "ZIP", "URL", "FISYR"]]

,WEBSITE,NAME,ADDRESS,ZIP,URL,FISYR
0,WWW.ROCHESTERBIZ.COM,GREATER ROCHESTER ENTERPRISE,100 CHESTNUT ONE HSBC PLAZA NO 1910,14604,https://s3.amazonaws.com/irs-form-990/20161124...,2015
1,WWW.LJBI.ORG,LA JOLLA BIOENGINEERING INSTITUTE,505 COAST BOULEVARD SOUTH,92037,https://s3.amazonaws.com/irs-form-990/20153349...,2014
2,NaN,ACTS COMMUNITY DEVELOPMENT,1034 66TH AVENUE,94621,https://s3.amazonaws.com/irs-form-990/20151313...,2014
3,NaN,AMERICAN AMISTAD FOUNDATION INC,733 SHINAVA DRIVE,84738,https://s3.amazonaws.com/irs-form-990/20151288...,2014
4,WWW.CASTLA.ORG,COALITION TO ABOLISH SLAVERY AND TRAFFICKING,5042 WILSHIRE BLVD NO 586,90036,https://s3.amazonaws.com/irs-form-990/20160137...,2014


In [13]:
float(len(data[pd.isnull(data.WEBSITE)])) / len(data)

0.2981337334708806

In [14]:
OkayWebsites = data[pd.isnull(data.WEBSITE) == False]
OkayWebsites.head(5)[["WEBSITE", "NAME", "ADDRESS", "ZIP", "URL", "FISYR"]]

,WEBSITE,NAME,ADDRESS,ZIP,URL,FISYR
0,WWW.ROCHESTERBIZ.COM,GREATER ROCHESTER ENTERPRISE,100 CHESTNUT ONE HSBC PLAZA NO 1910,14604,https://s3.amazonaws.com/irs-form-990/20161124...,2015
1,WWW.LJBI.ORG,LA JOLLA BIOENGINEERING INSTITUTE,505 COAST BOULEVARD SOUTH,92037,https://s3.amazonaws.com/irs-form-990/20153349...,2014
4,WWW.CASTLA.ORG,COALITION TO ABOLISH SLAVERY AND TRAFFICKING,5042 WILSHIRE BLVD NO 586,90036,https://s3.amazonaws.com/irs-form-990/20160137...,2014
5,HTTP://WALKRUNFORLIFE.COM/,ST LAWRENCE COUNTY CANCER FUND INC,PO BOX 291,13617,https://s3.amazonaws.com/irs-form-990/20164132...,2015
6,WWW.BRIDGTONHOSPITAL.ORG,BRIDGTON HOSPITAL,29 LOWELL STREET BOX A,4240,https://s3.amazonaws.com/irs-form-990/20160133...,2014


In [15]:
def exception_handler(request, exception):
    None
    #print "Request failed: " + str(request.url) + "; Exception: " + str(exception)

In [16]:
def ProcessText(text):
    soup = BeautifulSoup(text);
    return map(lambda link: link.get('href'), soup.find_all('a', attrs={'href': re.compile('twitter.com')}));

def ProcessRequest(requestsPerWebsite):
    #print(requestsPerWebsite);
    if (requestsPerWebsite[0] is not None and requestsPerWebsite[0].ok):
        return ProcessText(requestsPerWebsite[0].text);
    if (len(requestsPerWebsite) == 2):
        if (requestsPerWebsite[1] is not None and requestsPerWebsite[1].ok):
            return ProcessText(requestsPerWebsite[1].text);
    return [];

In [20]:
hdr = {'User-Agent': 'Mozilla/5.0'}
ishttp = re.compile('^http(s*):\/\/.*', flags=re.IGNORECASE)
def QueryWebsite(website):
    if (ishttp.match(website) is None):
        requests = ["http://" + website, "https://" + website];
    else:
        requests = [ website ];
    return list(grequests.get(u, timeout=4) for u in requests)

def ProcessWebsiteBatch(websiteBatch):
    unsentRequests = list(map(lambda x: QueryWebsite(x), websiteBatch));
    unsentRequestLengths = list(map(lambda x: len(x), unsentRequests));
    flattenedRequests = []
    for unsent in unsentRequests:
        flattenedRequests = flattenedRequests + unsent;
    
    print("Flattened Requests: " + str(len(flattenedRequests)))
    
    reqs = grequests.map(flattenedRequests, exception_handler=exception_handler)
    idx = 0;
    unFlattenedResponses = []
    for l in unsentRequestLengths:
        unFlattenedResponses = unFlattenedResponses + [reqs[idx : idx + l]];
        idx = idx + l;

    # print("Unflattened Requests: " + str(len(unFlattenedResponses)))
    
    return map(lambda x: ProcessRequest(x), unFlattenedResponses);

In [30]:
def BatchColumnRun(df, batchSize, csvwriter, csvfile):    
    finalNum = int(len(df) / batchSize);
    if (len(df) % batchSize != 0):
        finalNum = finalNum + 1
        
    for i in range(0, finalNum-1):
        division = df[i*batchSize:min((i+1)*batchSize, len(df))];
        twitterResults = ProcessWebsiteBatch(division["WEBSITE"]);
        for idx, twitterResult in enumerate(twitterResults):
            csv_writer.writerow([division["EIN"].values[idx], division["FISYR"].values[idx], twitterResult])
            csvfile.flush()
        print("Written: " + str(i * batchSize))


In [ ]:
with open('TwitterHandleOutput.csv', 'w') as csvfile:
    csv_writer = csv.writer(csvfile, delimiter='\t');
    # Write the header
    csv_writer.writerow(["EIN", "FISYR", "TWITTERHANDLE"])
    
    BatchColumnRun(OkayWebsites, 25, csv_writer, csvfile)
    csvfile.close()

Flattened Requests: 47
Written: 0
Flattened Requests: 50
Written: 25
Flattened Requests: 49
Written: 50
Flattened Requests: 48
Written: 75
Flattened Requests: 49
Written: 100
Flattened Requests: 48
Written: 125
Flattened Requests: 48
Written: 150
Flattened Requests: 49
Written: 175
Flattened Requests: 49
Written: 200
Flattened Requests: 50
Written: 225
Flattened Requests: 47
Written: 250
Flattened Requests: 48
Written: 275
Flattened Requests: 49
Written: 300
Flattened Requests: 50
Written: 325
Flattened Requests: 49
Written: 350
Flattened Requests: 50
Written: 375
Flattened Requests: 50
Written: 400
Flattened Requests: 49
Written: 425
Flattened Requests: 45
Written: 450
Flattened Requests: 47
Written: 475
Flattened Requests: 48
Written: 500
Flattened Requests: 49
Written: 525
Flattened Requests: 49
Written: 550
Flattened Requests: 48
Written: 575
Flattened Requests: 50
Written: 600
Flattened Requests: 48
Written: 625
Flattened Requests: 48
Written: 650
Flattened Requests: 49
Written: 6